In [1]:
from hourglass_tiny import HourglassModel
import os
import pandas as pd
import scipy.io
from scipy.misc import imresize
import matplotlib.pyplot as plt
from my_utils import *
import configparser

C:\Users\44163\AppData\Local\Programs\Python\Python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Using TensorFlow backend.


In [2]:
def process_config(conf_file):
	"""
	"""
	params = {}
	config = configparser.ConfigParser()
	config.read(conf_file)
	for section in config.sections():
		if section == 'DataSetHG':
			for option in config.options(section):
				params[option] = eval(config.get(section, option))
		if section == 'Network':
			for option in config.options(section):
				params[option] = eval(config.get(section, option))
		if section == 'Train':
			for option in config.options(section):
				params[option] = eval(config.get(section, option))
		if section == 'Validation':
			for option in config.options(section):
				params[option] = eval(config.get(section, option))
		if section == 'Saver':
			for option in config.options(section):
				params[option] = eval(config.get(section, option))
	return params
print('--Parsing Config File')
params = process_config('config.cfg')

--Parsing Config File


In [3]:
model = HourglassModel(nFeat=params['nfeats'], nStack=params['nstacks'], 
                       nModules=params['nmodules'], 
                       nLow=params['nlow'], outputDim=params['num_joints'], 
                       batch_size=params['batch_size'], attention = params['mcam'],
                       training=True, drop_rate= params['dropout_rate'], 
                       lear_rate=params['learning_rate'], 
                       decay=params['learning_rate_decay'], 
                       decay_step=params['decay_step'], 
                       name=params['name'], 
                       logdir_train=params['log_dir_train'], 
                       logdir_test=params['log_dir_test'], 
                       tiny= params['tiny'], w_loss=params['weighted_loss'] , 
                       joints= params['joint_list'],modif=False)
model.generate_model()

CREATE MODEL:
---Inputs : Done (0 sec.)


---Graph : Done (11 sec.)
---Loss : Done (0 sec.)


In [4]:
# last map: (None,64,64,15)
output_stack = model.output
stacks = tf.unstack(output_stack,axis=1)
last_map = stacks[3]
# add layers for upsampling, from (64,64,15) to (256,256,15)
bi_upsampling = tf.image.resize_bilinear(last_map,[256,256],name='bilinear_upsampling')
W_last = weight_variable([3,3,15,15],name='W_last')
b_last = bias_variable([15],name='b_last')
aft_upsam_out = conv2d_basic(bi_upsampling,W_last,b_last)
saver = tf.train.Saver()
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
path = "trained_nets/net_with_pascal_Mar_27_upsampling_2"
full_path = os.path.join(path,'save_net.ckpt')
saver.restore(sess,full_path)

INFO:tensorflow:Restoring parameters from trained_nets/net_with_pascal_Mar_27_upsampling_2\save_net.ckpt


In [5]:
pascal_data = pd.read_csv('pacal_data_inform.csv')
pascal_data.head()

,Unnamed: 0,jpg_path,mat_path
0,0,VOC2010/JPEGImages\2008_000003.jpg,VOC2010/all_person_masks\2008_000003.mat
1,1,VOC2010/JPEGImages\2008_000008.jpg,VOC2010/all_person_masks\2008_000008.mat
2,2,VOC2010/JPEGImages\2008_000023.jpg,VOC2010/all_person_masks\2008_000023.mat
3,3,VOC2010/JPEGImages\2008_000026.jpg,VOC2010/all_person_masks\2008_000026.mat
4,4,VOC2010/JPEGImages\2008_000032.jpg,VOC2010/all_person_masks\2008_000032.mat


In [17]:
i = 3430
img = image.img_to_array(image.load_img(pascal_data['jpg_path'][i]))
label = scipy.io.loadmat(pascal_data['mat_path'][i])['single_map']
cut1,cut2,cut3 = random_put_into_256map(img,label)
plt.figure()
plt.imshow(cut3[:,:,0:3]/256)
plt.show()
plt.figure()
plt.imshow(cut3[:,:,3])
plt.show()
test_image = np.expand_dims(cut3[:,:,0:3],0)
output = sess.run(aft_upsam_out,feed_dict={model.img:test_image})
output = np.reshape(output,[256,256,15])
output = np.argmax(output,axis=2)
plt.figure()
plt.imshow(output)
plt.show()


In [6]:
acc_list=[]
iou_list=[]
for i in range(3490,3500):
    img = image.img_to_array(image.load_img(pascal_data['jpg_path'][i]))
    label = scipy.io.loadmat(pascal_data['mat_path'][i])['single_map']
    cut1,cut2,cut3 = random_put_into_256map(img,label)
    test_image = np.expand_dims(cut3[:,:,0:3],0)
    output = sess.run(aft_upsam_out,feed_dict={model.img:test_image})
    output = np.reshape(output,[256,256,15])
    output = np.argmax(output,axis=2)
    temp_acc = cal_acc(output,cut3[:,:,3])
    if temp_acc != 0:
        acc_list.append(temp_acc)
    temp_iou = cal_iou_for_upsamp(output,cut3[:,:,3])
    if temp_iou != 0:
        iou_list.append(temp_iou)

In [11]:
np.mean(iou_list)

0.1768417777951075

In [14]:
sitting_data = pd.read_csv('sitting_data_table.csv')
sitting_data.head()

,Unnamed: 0,data_name,label_name,data_addr,label_addr
0,0,image0.jpg,image0.mat,Sitting/img\image0.jpg,Sitting/masks\image0.mat
1,1,image1.jpg,image1.mat,Sitting/img\image1.jpg,Sitting/masks\image1.mat
2,2,image10.jpg,image10.mat,Sitting/img\image10.jpg,Sitting/masks\image10.mat
3,3,image100.jpg,image100.mat,Sitting/img\image100.jpg,Sitting/masks\image100.mat
4,4,image101.jpg,image101.mat,Sitting/img\image101.jpg,Sitting/masks\image101.mat


In [20]:
i = 23
test_img = image.load_img(sitting_data['data_addr'][i])
test_img = image.img_to_array(test_img)
test_mask = scipy.io.loadmat(sitting_data['label_addr'][i])['M']
cut1,cut2,cut3 = random_put_into_256map(test_img,test_mask)
plt.figure()
plt.imshow(cut3[:,:,0:3]/256)
plt.show()
plt.figure()
plt.imshow(cut3[:,:,3])
plt.show()
test_image = np.expand_dims(cut3[:,:,0:3],0)
output = sess.run(aft_upsam_out,feed_dict={model.img:test_image})
output = np.reshape(output,[256,256,15])
output = np.argmax(output,axis=2)
plt.figure()
plt.imshow(output)
plt.show()

In [22]:
acc_list=[]
iou_list=[]
for i in range(80,101):
    img = image.img_to_array(image.load_img(pascal_data['data_addr'][i]))
    label = scipy.io.loadmat(pascal_data['label_addr'][i])['M']
    cut1,cut2,cut3 = random_put_into_256map(img,label)
    test_image = np.expand_dims(cut3[:,:,0:3],0)
    output = sess.run(aft_upsam_out,feed_dict={model.img:test_image})
    output = np.reshape(output,[256,256,15])
    output = np.argmax(output,axis=2)
    temp_acc = cal_acc(output,cut3[:,:,3])
    if temp_acc != 0:
        acc_list.append(temp_acc)
    temp_iou = cal_iou_for_upsamp(output,cut3[:,:,3])
    if temp_iou != 0:
        iou_list.append(temp_iou)

In [24]:
np.mean(iou_list)

0.19615659111699574